In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, Tuple

import torch
import torch.nn.functional as F
from pytorch_lightning import LightningModule
from torch import Tensor, nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import MNIST

from energizer import DeterministicMixin, MCDropoutMixin, Trainer
from energizer.acquisition_functions import entropy, expected_entropy
from energizer.query_strategies.base import AccumulatorStrategy, RandomStrategy

In [3]:
# load and preprocess datasets
data_dir = "./data"
preprocessing_pipe = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)
train_set = MNIST(data_dir, train=True, download=False, transform=preprocessing_pipe)
test_set = MNIST(data_dir, train=False, download=False, transform=preprocessing_pipe)
train_set, val_set = random_split(train_set, [55000, 5000])

# create dataloaders
batch_size = 32
eval_batch_size = 32  # this is use when evaluating on the pool too
train_dl = DataLoader(train_set, batch_size=batch_size)
val_dl = DataLoader(val_set, batch_size=eval_batch_size)
test_dl = DataLoader(test_set, batch_size=eval_batch_size)

In [4]:
class MNISTModel(LightningModule):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(1024, 128),
            nn.Dropout(),
            nn.Linear(128, 10),
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.model(x)

    def loss(self, logits: Tensor, targets: Tensor) -> Tensor:
        return F.cross_entropy(logits, targets)

    def step(self, batch: Tuple[Tensor, Tensor], stage: str) -> Dict[str, Tensor]:
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        self.log(f"{stage}/loss", loss)
        return {"loss": loss, "logits": logits}

    def training_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Dict[str, Tensor]:
        return self.step(batch, "train")

    def validation_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Dict[str, Tensor]:
        return self.step(batch, "val")

    def test_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Dict[str, Tensor]:
        return self.step(batch, "test")

    def configure_optimizers(self) -> None:
        return torch.optim.SGD(self.parameters(), lr=0.01)


class DeterministicMNISTModel(DeterministicMixin):
    """A implememntation of the `Entropy` active learning strategy."""

    def pool_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Tensor:
        # define how to perform the forward pass
        x, _ = batch
        logits = self(x)
        # use an acquisition/scoring function
        scores = entropy(logits)
        return scores


class StochasticMNISTModel(MCDropoutMixin):
    """A implememntation of the `Entropy` active learning strategy.

    In this case we use the MCDropout technique to compute model
    uncertainty. Accordigly, we need to use `expected_entropy` as
    the acquisition function.
    """

    def loss(logits: Tensor, targets: Tensor) -> Tensor:
        # logits has now shape:
        # [num_samples, num_classes, num_iterations]
        # average over num_iterations
        logits = logits.mean(-1)
        return F.cross_entropy(logits, targets)

    def pool_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Tensor:
        """A implememntation of the `Entropy` active learning strategy."""
        # define how to perform the forward pass
        x, _ = batch
        logits = self(x)
        # use an acquisition/scoring function
        # NOTE: since we are using MCDropout we need to use the
        # `expected_entropy` acquisition function
        scores = expected_entropy(logits)
        return scores

In [5]:
class EntropyStrategy(AccumulatorStrategy):
    """A implememntation of the `Entropy` active learning strategy."""

    def pool_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Tensor:
        # define how to perform the forward pass
        x, _ = batch
        logits = self(x)
        # use an acquisition/scoring function
        scores = entropy(logits)
        return scores

In [6]:
model = MNISTModel()
deterministic_model = DeterministicMNISTModel(model)
stochastic_model = StochasticMNISTModel(model)
entropy_strategy = EntropyStrategy(model)
random_strategy = RandomStrategy(model)

In [7]:
x, _ = next(iter(train_dl))
model(x).shape, deterministic_model(x).shape, stochastic_model(x).shape, entropy_strategy(x).shape, random_strategy(x).shape

(torch.Size([32, 10]),
 torch.Size([32, 10]),
 torch.Size([32, 10, 10]),
 torch.Size([32, 10]),
 torch.Size([32, 10]))

In [8]:
trainer = Trainer(
    query_size=2,
    max_epochs=3,
    max_labelling_epochs=4,
    total_budget=5,
    log_every_n_steps=1,
    test_after_labelling=True,
    # for testing purposes
    limit_train_batches=10,
    limit_val_batches=10,
    limit_test_batches=10,
    limit_pool_batches=10,
)

trainer.active_fit(
    model=entropy_strategy,
    train_dataloaders=train_dl,
    val_dataloaders=val_dl,
    test_dataloaders=test_dl,
)

trainer.datamodule.stats

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 184 K 
-------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Using `BaseQueryStrategy`
Using `BaseQueryStrategy`
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, pool_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Pool DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 41.76it/s]

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric AccumulateTopK was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
Using underlying `LightningModule`


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Epoch 2: 100%|██████████| 11/11 [00:00<00:00, 31.59it/s, loss=1.76, v_num=14]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 11/11 [00:00<00:00, 30.96it/s, loss=1.76, v_num=14]

Using `BaseQueryStrategy`


{'total_data_size': 55000,
 'train_size': 4,
 'pool_size': 54996,
 'num_train_batches': 1,
 'num_pool_batches': 1719}

In [9]:
trainer = Trainer(
    query_size=2,
    max_epochs=3,
    max_labelling_epochs=4,
    total_budget=5,
    log_every_n_steps=1,
    test_after_labelling=True,
    # for testing purposes
    limit_train_batches=10,
    limit_val_batches=10,
    limit_test_batches=10,
    limit_pool_batches=10,
)

trainer.active_fit(
    model=random_strategy,
    train_dataloaders=train_dl,
    val_dataloaders=val_dl,
    test_dataloaders=test_dl,
)

trainer.datamodule.stats

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 184 K 
-------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


Using `BaseQueryStrategy`
Using `BaseQueryStrategy`
Using underlying `LightningModule`


Epoch 2: 100%|██████████| 11/11 [00:00<00:00, 37.27it/s, loss=2.06, v_num=15]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 11/11 [00:00<00:00, 35.63it/s, loss=2.06, v_num=15]

...

Epoch 2: 100%|██████████| 11/11 [00:00<00:00, 35.78it/s, loss=2.15, v_num=15]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 11/11 [00:00<00:00, 34.90it/s, loss=2.15, v_num=15]

Using `BaseQueryStrategy`


{'total_data_size': 55000,
 'train_size': 6,
 'pool_size': 54994,
 'num_train_batches': 1,
 'num_pool_batches': 1719}